# [3주차 과제(9.19 업로드)](!https://cyber.gachon.ac.kr/mod/ubboard/article.php?id=859585&page=2&bwid=789787)

- Abalone 데이터 CNN으로 분류(Classification), 회귀(Regression) 작성(2주차 과제에서 진행함)
- Abalone 데이터 LSTM으로 분류(Classification), 회귀(Regression) 작성

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from scipy import stats
from sklearn.model_selection import train_test_split

file_path = './abalone.csv'

In [2]:
df = pd.read_csv(file_path, header = 0) # header가 존재하기에, 0번째 행을 header로 지정
df = df.drop("id", axis = 1) # id 열을 지울거니까 axis를 1로 지정 (*행을 지우려면 axis = 1)
df

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [3]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

df['Sex'] = label.fit_transform(df['Sex'])
df.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,2,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Abalone 데이터 LSTM으로 분류(Classification), 회귀(Regression) 작성

In [5]:
df['AgeGroup'] = pd.cut(df['Rings'], bins=[0, 10, 20, float('inf')], labels=[0, 1, 2])  # Young, Middle-aged, Old
X = df.drop(['Rings', 'AgeGroup'], axis=1)
y = df['AgeGroup']

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X = X.reshape((X.shape[0], 1, X.shape[1]))
y = to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: Young, Middle-aged, Old
])

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

Epoch 1/50
167/167 [==============================] - 1s 3ms/step - loss: 0.7170 - accuracy: 0.7100 - val_loss: 0.6044 - val_accuracy: 0.7130
Epoch 2/50
167/167 [==============================] - 0s 2ms/step - loss: 0.5598 - accuracy: 0.7335 - val_loss: 0.5590 - val_accuracy: 0.7145
Epoch 3/50
167/167 [==============================] - 0s 2ms/step - loss: 0.5279 - accuracy: 0.7466 - val_loss: 0.5226 - val_accuracy: 0.7623
Epoch 4/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4975 - accuracy: 0.7601 - val_loss: 0.4971 - val_accuracy: 0.7803
Epoch 5/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4833 - accuracy: 0.7725 - val_loss: 0.4917 - val_accuracy: 0.7848
Epoch 6/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4730 - accuracy: 0.7788 - val_loss: 0.4906 - val_accuracy: 0.7848
Epoch 7/50
167/167 [==============================] - 0s 2ms/step - loss: 0.4673 - accuracy: 0.7773 - val_loss: 0.4983 - val_accuracy: 0.7848
Epoch 

In [11]:
# Evaluate
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("\n--- LSTM Classification on Abalone ---")
print("Test Accuracy:", test_accuracy)

27/27 [==============================] - 0s 808us/step - loss: 0.4876 - accuracy: 0.7787

--- LSTM Classification on Abalone ---
Test Accuracy: 0.7787081599235535
